# **PROYECTO SQL - Analítica en la base de datos - NSLDINA**  

## **Alejandro Fernández Rodríguez -** **<u>Máster Data Science EFBS Edición 2021</u>**

En el presente proyecto, que parte de un análisis integral de carácter financiero (ventas, ingresos, costes y gastos) cuya finalidad es obtener una cuenta de resultados que abarque los doce meses de los que ostentamos datos, se pretende:

>  <span style="background-color: rgba(127, 127, 127, 0.1);">&nbsp;· Ofrecer una valoración, a partir de lo que nos ofrezcan los datos, acerca del desempeño económico del proyecto NSLDINA en su primer año de vigencia.</span>
> 
> · A partir de la cuenta de resultados y el análisis de la facturación, establecer una estrategia consistente en una serie de puntos en los que consideremos que la empresa puede incidir para poder hacer frente a sus debilidades y mejorar los guarismos en los indicadores clave de la cuenta de resultados a corto/medio plazo.

Para poder valorar con cierta viabilidad y justicia el desempeño del proyecto y, ante la ausencia, en algunos puntos, de ciertos datos clave, se parte de los siguientes supuestos:

>   · Las áreas existentes en la tabla DIM\_ZONA cubren sólo terreno terrestre de una extensión total de unos 20.000 km. cuadrados (medidos mediante un mapa usando las coordenadas reales -**VER AREA\_TOTAL.PNG en ANEXOS**\-).
> 
>   · Se establece un coste fijo mensual para los rentings de vehículo (uno por comercial) de 250 EUR/mes. En el enunciado del ejercicio, en el cual se presentan las características de la empresa, se menciona la existencia de estos vehículos, pero no así en el caso de los riders. Asimismo, se omiten otros posibles costes fijos existentes a los que no se hace mención en el enunciado, aunque posiblemente existirían: suministros, alquiler oficina, sistemas y mantenimiento informático, dietas, etc, puesto que el objetivo principal del proyecto no es la elaboración de un plan empresarial desde el punto de vista financiero.
> 
>   · Se establece el precio de venta de las ensaladas en 16 EUR para clientes no suscritos, mientras que para los suscritos serían 9 EUR hasta llegar a la cantidad comprometida. A partir de ahí, el precio sube a 12 EUR.
> 
>  · Independientemente de si el pedido se hace con o sin suscripción, **se entenderá que las ensaladas se envían a cliente cada vez que se solicitan (tabla FACT\_VENTA)**, y no que todas las ensaladas de la suscripción se envían a la vez. Para tratar de estimar cuantas ensaladas solicita cada cliente cada vez que pide se elaboran unos coeficientes simples (explicados en el siguiente bloque de texto), que nos ayudan a cuantificar los costes totales de entrega.

## **SUPUESTOS PREVIOS + ANÁLISIS EXPLORATORIO**

In [1]:
USE NSLDINA;

SELECT COUNT(*) FROM FACT_VENTA;
SELECT COUNT(DISTINCT CLIENTE_ID) AS NUM_CLIENTES_DISTINTOS_SUSCRITOS FROM FACT_SUBSCRIPCION;
SELECT COUNT(DISTINCT CLIENTE_ID) AS NUM_CLIENTES_DISTINTOS FROM FACT_VENTA;
SELECT DISTINCT MES_ID,
                COUNT(CLIENTE_ID) OVER (PARTITION BY MES_ID) AS CLIENTES_SUSCRITOS,
                SUM(COMPROMISO) OVER (PARTITION BY MES_ID) AS ENSALADAS
FROM FACT_SUBSCRIPCION;

SELECT DISTINCT EMPLEADO_ID, IMPORTE_NOMINA FROM FACT_NOMINA;

SELECT COUNT(*) FROM DIM_CLIENTE;
SELECT * FROM DIM_COMERCIAL;
SELECT * FROM DIM_ZONA

(1 row affected)

(1 row affected)

(1 row affected)

(12 rows affected)

(7 rows affected)

...

MES_ID,CLIENTES_SUSCRITOS,ENSALADAS
201905,41,557
201906,76,921
201907,108,1330
201908,132,1632
201909,173,1930
201910,201,2068
201911,235,2369
201912,269,2649
202001,306,3219
202002,339,3623


EMPLEADO_ID,IMPORTE_NOMINA
COCINERO - 01,1176
COMERCIAL 115,1197
COMERCIAL 116,1862
COMERCIAL 121,"1117,2"
COMERCIAL 129,1197
COMERCIAL 135,1197
COMERCIAL 137,"1303,4"


(No column name)
436


POSICIONABLE_ID,COMERCIAL_ID,SALARIO_BASE,COMISION_CLIENTE
115,COMERCIAL 115,900,12
116,COMERCIAL 116,1400,0
121,COMERCIAL 121,840,16
129,COMERCIAL 129,900,12
135,COMERCIAL 135,900,12
137,COMERCIAL 137,980,10


ZONA_ID,LATITUD_GRADO,LONGITUD_GRADO
ZONA - 1,31,-9
ZONA - 2,32,-9
ZONA - 3,32,-8
ZONA - 4,33,-9
ZONA - 5,33,-8
ZONA - 6,34,-8


## **ANÁLISIS FACTURACIÓN, COSTES Y RESULTADO**

El primer paso del proyecto es realizar un análisis detallado de los ingresos y costes, a partir de los datos de facturación de los que disponemos. Para cliente en cada mes hallaremos los ingresos y costes operativos (producto y entrega) inherentes a cada línea de venta (suscripción o fuera de suscripción -EXTRA-). Posteriormente agregaremos a esos datos los costes no procedentes directamente de la explotación -personal y renting, en este caso- y obtendremos las métricas básicas de la cuenta de resultados, así como una composición de costes cuyo análisis nos permitirá identificar y construir un plan de actuación a corto y medio plazo.

Para poder cuantificar los costes de entrega, fijamos un coeficiente que nos sirva para estimar la cantidad de envíos mensuales (cada envío tiene un coste de 1,5 EUR) que se realiza a cada cliente. Por ello, se entiende que, para aquellas suscripciones que contengan, mensualmente, una cantidad de 10 ensaladas o menos, los envíos siempre contendrán una sola ensalada (coeficiente 1). Cada ensalada servida a ese cliente en ese mes tendría un coste de 1,5EUR \* 1 = 1,5EUR.

En caso de comprometerse a +10 ensaladas o menos de 20, se entendería que existen dos personas suscritas dentro de esa cuenta de cliente, y que al decidir consumir una ensalada realmente solicitan dos (una para cada uno) cada vez que piden. Por tanto, multiplicaremos \* 0,5 el total de ensaladas que consuma ese cliente en un mes (cada envío lleva dos ensaladas). Cada ensalada servida a ese cliente costaría 1,5EUR\*0,5 = 1EUR. En caso de que el compromiso sea superior a 20 ensaladas se entiende que ese cliente es en realidad una unidad familiar (o un grupo de compañeros de piso/oficina). Se le aplica un coeficiente inferior (0,25) dado que estimamos que cada pedido puede contener 3, 4 o 5 ensaladas.

In [2]:
SELECT TOP 10 *, CASE WHEN COMPROMISO <=10 THEN 1 WHEN COMPROMISO BETWEEN 10 AND 20 THEN 0.5 ELSE 0.25 END AS COEFICIENTE FROM FACT_SUBSCRIPCION

(10 rows affected)

Total execution time: 00:00:00.098

CLIENTE_ID,SEXO_ID,MES_ID,COMPROMISO,COEFICIENTE
Cliente - 001,HOMBRE,201908,12,"0,50"
Cliente - 001,HOMBRE,201909,12,"0,50"
Cliente - 001,HOMBRE,201910,7,"1,00"
Cliente - 001,HOMBRE,201911,10,"1,00"
Cliente - 001,HOMBRE,201912,11,"0,50"
Cliente - 001,HOMBRE,202001,15,"0,50"
Cliente - 001,HOMBRE,202002,18,"0,50"
Cliente - 001,HOMBRE,202003,11,"0,50"
Cliente - 001,HOMBRE,202004,9,"1,00"
Cliente - 002,HOMBRE,202003,8,"1,00"


In [3]:
SELECT DISTINCT TOP 10
    CLIENTE_ID, 
    COUNT(*) OVER (PARTITION BY CLIENTE_ID) AS CUENTA,
    SUM(COMPROMISO) OVER (PARTITION BY CLIENTE_ID) AS ENSALADAS_COMPRADAS_MES,
    SUM(COMPROMISO) OVER (PARTITION BY CLIENTE_ID) * COUNT(*) OVER (PARTITION BY CLIENTE_ID) AS TOTAL_ENSALADAS,
    SUM(COMPROMISO) OVER (PARTITION BY CLIENTE_ID) * COUNT(*) OVER (PARTITION BY CLIENTE_ID) * 9 AS INGRESOS_SUSCRIPCION
FROM FACT_SUBSCRIPCION
ORDER BY TOTAL_ENSALADAS DESC

(10 rows affected)

Total execution time: 00:00:00.140

CLIENTE_ID,CUENTA,ENSALADAS_COMPRADAS_MES,TOTAL_ENSALADAS,INGRESOS_SUSCRIPCION
Cliente - 066,12,172,2064,18576
Cliente - 240,12,166,1992,17928
Cliente - 275,12,166,1992,17928
Cliente - 400,12,165,1980,17820
Cliente - 116,12,157,1884,16956
Cliente - 290,12,154,1848,16632
Cliente - 337,12,153,1836,16524
Cliente - 190,12,152,1824,16416
Cliente - 201,12,151,1812,16308
Cliente - 084,12,150,1800,16200


In [4]:
SELECT TOP 10 * FROM FACT_VENTA V
LEFT JOIN DIM_PRODUCTO P 
    ON V.PRODUCTO_ID = P.PRODUCTO_ID

(10 rows affected)

Total execution time: 00:00:00.105

CLIENTE_ID,MES_ID,PRODUCTO_ID,CANTIDAD,PRODUCTO_ID,COSTE
Cliente - 001,201908,Ensalada - 01,3,Ensalada - 01,"3,5"
Cliente - 001,201908,Ensalada - 04,3,Ensalada - 04,"4,4"
Cliente - 001,201908,Ensalada - 05,2,Ensalada - 05,"4,7"
Cliente - 001,201908,Ensalada - 08,2,Ensalada - 08,"5,1"
Cliente - 001,201908,Ensalada - 09,2,Ensalada - 09,"5,4"
Cliente - 001,201908,Ensalada - 12,2,Ensalada - 12,6
Cliente - 001,201908,Ensalada - 15,3,Ensalada - 15,"7,4"
Cliente - 001,201909,Ensalada - 02,1,Ensalada - 02,"3,8"
Cliente - 001,201909,Ensalada - 07,1,Ensalada - 07,"5,1"
Cliente - 001,201909,Ensalada - 08,1,Ensalada - 08,"5,1"


In [5]:
WITH VENTA_COSTE AS (
    SELECT V.CLIENTE_ID,V.PRODUCTO_ID,V.MES_ID,V.CANTIDAD,P.COSTE 
    FROM FACT_VENTA V
LEFT JOIN DIM_PRODUCTO P 
    ON V.PRODUCTO_ID = P.PRODUCTO_ID
),

SUSCRIPCION AS (
    SELECT *, CASE WHEN COMPROMISO <=10 THEN 1 WHEN COMPROMISO BETWEEN 10 AND 20 THEN 0.5 ELSE 0.25 END AS COEFICIENTE 
    FROM FACT_SUBSCRIPCION
)

SELECT DISTINCT
    V.CLIENTE_ID,
    V.MES_ID,
    S.COEFICIENTE AS COEFICIENTE_COSTE_ENTREGA,
    S.COMPROMISO AS SUSCRIPCION,
    SUM(CANTIDAD) OVER (PARTITION BY V.CLIENTE_ID,V.MES_ID) - S.COMPROMISO AS EXTRA,
    SUM(CANTIDAD) OVER (PARTITION BY V.CLIENTE_ID,V.MES_ID) AS TOTAL,
    S.COMPROMISO * 9 AS INGRESOS_SUSCRIPCION_MES,
    S.COMPROMISO * S.COEFICIENTE * 1.5 AS COSTE_ENTREGA_SUB_MES,
    S.COMPROMISO * AVG(V.COSTE) OVER (PARTITION BY V.CLIENTE_ID,V.MES_ID) AS COSTE_SUSCRIPCION_MES,
    CASE WHEN S.COMPROMISO = 0 THEN (SUM(CANTIDAD) OVER (PARTITION BY V.CLIENTE_ID,V.MES_ID)) * 16 ELSE (SUM(CANTIDAD) OVER (PARTITION BY V.CLIENTE_ID,V.MES_ID) - S.COMPROMISO) * 12 END AS INGRESOS_EXTRA_MES,
    (SUM(CANTIDAD) OVER (PARTITION BY V.CLIENTE_ID,V.MES_ID) - S.COMPROMISO) * S.COEFICIENTE * 1.5 AS COSTE_ENTREGA_EXTRA_MES,
    (SUM(CANTIDAD) OVER (PARTITION BY V.CLIENTE_ID,V.MES_ID) - S.COMPROMISO) * AVG(V.COSTE) OVER (PARTITION BY V.CLIENTE_ID,V.MES_ID) AS COSTE_EXTRA_MES,
    CASE WHEN S.COMPROMISO = 0 THEN (SUM(CANTIDAD) OVER (PARTITION BY V.CLIENTE_ID,V.MES_ID)) * 16 ELSE (S.COMPROMISO * 9) + (SUM(CANTIDAD) OVER (PARTITION BY V.CLIENTE_ID,V.MES_ID) - S.COMPROMISO) * 12 END AS INGRESOS_TOTALES_MES,
    SUM(CANTIDAD) OVER (PARTITION BY V.CLIENTE_ID,V.MES_ID) * S.COEFICIENTE * 1.5 AS COSTE_ENTREGA_MES,
    (SUM(CANTIDAD) OVER (PARTITION BY V.CLIENTE_ID,V.MES_ID) * AVG(V.COSTE) OVER (PARTITION BY V.CLIENTE_ID,V.MES_ID)) AS COSTE_TOTAL_MES
FROM VENTA_COSTE V
LEFT JOIN SUSCRIPCION S 
    ON V.CLIENTE_ID = S.CLIENTE_ID AND S.MES_ID = V.MES_ID

(2653 rows affected)

Total execution time: 00:00:01.706

CLIENTE_ID,MES_ID,COEFICIENTE_COSTE_ENTREGA,SUSCRIPCION,EXTRA,TOTAL,INGRESOS_SUSCRIPCION_MES,COSTE_ENTREGA_SUB_MES,COSTE_SUSCRIPCION_MES,INGRESOS_EXTRA_MES,COSTE_ENTREGA_EXTRA_MES,COSTE_EXTRA_MES,INGRESOS_TOTALES_MES,COSTE_ENTREGA_MES,COSTE_TOTAL_MES
Cliente - 001,201908,"0,50",12,5,17,108,"9,000","62,57142857142857",60,"3,750","26,071428571428573",168,"12,750","88,64285714285714"
Cliente - 001,201909,"0,50",12,0,12,108,"9,000","66,17142857142858",0,"0,000",0,108,"9,000","66,17142857142858"
Cliente - 001,201910,"1,00",7,0,7,63,"10,500","40,459999999999994",0,"0,000",0,63,"10,500","40,459999999999994"
Cliente - 001,201911,"1,00",10,0,10,90,"15,000","57,57142857142857",0,"0,000",0,90,"15,000","57,57142857142857"
Cliente - 001,201912,"0,50",11,0,11,99,"8,250","59,18",0,"0,000",0,99,"8,250","59,18"
Cliente - 001,202001,"0,50",15,0,15,135,"11,250","85,875",0,"0,000",0,135,"11,250","85,875"
Cliente - 001,202002,"0,50",18,0,18,162,"13,500","101,925",0,"0,000",0,162,"13,500","101,925"
Cliente - 001,202003,"0,50",11,5,16,99,"8,250","58,771428571428565",60,"3,750","26,71428571428571",159,"12,000","85,48571428571428"
Cliente - 001,202004,"1,00",9,3,12,81,"13,500","48,42",36,"4,500","16,14",117,"18,000","64,56"
Cliente - 002,202003,"1,00",8,1,9,72,"12,000","49,06666666666666",12,"1,500","6,133333333333333",84,"13,500","55,199999999999996"


In [6]:
DECLARE @COSTE_NOMINA AS NUMERIC
SET @COSTE_NOMINA = (SELECT DISTINCT SUM(IMPORTE_NOMINA) OVER (PARTITION BY MES_ID) FROM FACT_NOMINA)
DECLARE @COSTE_RENTING AS INT = 250;

WITH VENTA_COSTE AS (
    SELECT V.CLIENTE_ID,V.PRODUCTO_ID,V.MES_ID,V.CANTIDAD,P.COSTE FROM FACT_VENTA V
LEFT JOIN DIM_PRODUCTO P 
    ON V.PRODUCTO_ID = P.PRODUCTO_ID
),

SUSCRIPCION AS (
    SELECT *, CASE WHEN COMPROMISO <=10 THEN 1 WHEN COMPROMISO BETWEEN 10 AND 20 THEN 0.5 ELSE 0.25 END AS COEFICIENTE FROM FACT_SUBSCRIPCION
),

FACTURACION AS (
SELECT DISTINCT
    V.CLIENTE_ID,
    V.MES_ID,
    S.COEFICIENTE AS COEFICIENTE_COSTE_ENTREGA,
    S.COMPROMISO AS SUSCRIPCION,
    SUM(CANTIDAD) OVER (PARTITION BY V.CLIENTE_ID,V.MES_ID) - S.COMPROMISO AS EXTRA,
    SUM(CANTIDAD) OVER (PARTITION BY V.CLIENTE_ID,V.MES_ID) AS TOTAL,
    S.COMPROMISO * 9 AS INGRESOS_SUSCRIPCION_MES,
    S.COMPROMISO * S.COEFICIENTE * 1.5 AS COSTE_ENTREGA_SUB_MES,
    S.COMPROMISO * AVG(V.COSTE) OVER (PARTITION BY V.CLIENTE_ID,V.MES_ID) AS COSTE_SUSCRIPCION_MES,
    CASE WHEN S.COMPROMISO = 0 THEN (SUM(CANTIDAD) OVER (PARTITION BY V.CLIENTE_ID,V.MES_ID)) * 16 ELSE (SUM(CANTIDAD) OVER (PARTITION BY V.CLIENTE_ID,V.MES_ID) - S.COMPROMISO) * 12 END AS INGRESOS_EXTRA_MES,
    (SUM(CANTIDAD) OVER (PARTITION BY V.CLIENTE_ID,V.MES_ID) - S.COMPROMISO) * S.COEFICIENTE * 1.5 AS COSTE_ENTREGA_EXTRA_MES,
    (SUM(CANTIDAD) OVER (PARTITION BY V.CLIENTE_ID,V.MES_ID) - S.COMPROMISO) * AVG(V.COSTE) OVER (PARTITION BY V.CLIENTE_ID,V.MES_ID) AS COSTE_EXTRA_MES,
    CASE WHEN S.COMPROMISO = 0 THEN (SUM(CANTIDAD) OVER (PARTITION BY V.CLIENTE_ID,V.MES_ID)) * 16 ELSE (S.COMPROMISO * 9) + (SUM(CANTIDAD) OVER (PARTITION BY V.CLIENTE_ID,V.MES_ID) - S.COMPROMISO) * 12 END AS INGRESOS_TOTALES_MES,
    SUM(CANTIDAD) OVER (PARTITION BY V.CLIENTE_ID,V.MES_ID) * S.COEFICIENTE * 1.5 AS COSTE_ENTREGA_MES,
    (SUM(CANTIDAD) OVER (PARTITION BY V.CLIENTE_ID,V.MES_ID) * AVG(V.COSTE) OVER (PARTITION BY V.CLIENTE_ID,V.MES_ID)) AS COSTE_TOTAL_MES
FROM VENTA_COSTE V
LEFT JOIN SUSCRIPCION S 
    ON V.CLIENTE_ID = S.CLIENTE_ID AND S.MES_ID = V.MES_ID
)


SELECT SUM(INGRESOS_SUSCRIPCION_MES) AS INGRESOS_SUSCRIPCION,
        SUM(COSTE_SUSCRIPCION_MES+COSTE_ENTREGA_SUB_MES) AS COSTE_SUSCRIPCION,
        (SUM(INGRESOS_SUSCRIPCION_MES) - SUM(COSTE_SUSCRIPCION_MES+COSTE_ENTREGA_SUB_MES))/SUM(INGRESOS_SUSCRIPCION_MES) AS MARGENBRUTO_SUSCRIPCION,
        SUM(INGRESOS_EXTRA_MES) AS INGRESOS_EXTRA,
        SUM(COSTE_EXTRA_MES+COSTE_ENTREGA_EXTRA_MES) AS COSTE_EXTRA,
        (SUM(INGRESOS_EXTRA_MES) - SUM(COSTE_EXTRA_MES+COSTE_ENTREGA_EXTRA_MES))/SUM(INGRESOS_EXTRA_MES) AS MARGENBRUTO_EXTRA,
        SUM(INGRESOS_TOTALES_MES) AS INGRESOS_TOTALES,
        SUM(COSTE_TOTAL_MES+COSTE_ENTREGA_MES) AS COSTES_TOTALES_BRUTO,
        SUM(COSTE_ENTREGA_MES) AS COSTE_ENTREGA,
        @COSTE_NOMINA*12 + @COSTE_RENTING*6*12 AS COSTE_PERSONAL,
        SUM(COSTE_TOTAL_MES+COSTE_ENTREGA_MES) + @COSTE_RENTING*6*12 + @COSTE_NOMINA*12 AS COSTES_TOTALES,
        (SUM(INGRESOS_TOTALES_MES) - SUM(COSTE_TOTAL_MES))/SUM(INGRESOS_TOTALES_MES) AS MARGEN_BRUTO,
        (SUM(INGRESOS_TOTALES_MES) - (@COSTE_NOMINA*12 + @COSTE_RENTING*6*12 + SUM(COSTE_TOTAL_MES+COSTE_ENTREGA_MES))) AS RESULTADO_OPERATIVORESULTADO_OPERATIVO,
        (SUM(INGRESOS_TOTALES_MES) - (@COSTE_NOMINA*12 + @COSTE_RENTING*6*12 + SUM(COSTE_TOTAL_MES+COSTE_ENTREGA_MES)))/SUM(INGRESOS_TOTALES_MES) AS MARGEN_NETO
FROM FACTURACION;

WITH VENTA_COSTE AS (
    SELECT V.CLIENTE_ID,V.PRODUCTO_ID,V.MES_ID,V.CANTIDAD,P.COSTE FROM FACT_VENTA V
LEFT JOIN DIM_PRODUCTO P 
    ON V.PRODUCTO_ID = P.PRODUCTO_ID
),

SUSCRIPCION AS (
    SELECT *, CASE WHEN COMPROMISO <=10 THEN 1 WHEN COMPROMISO BETWEEN 10 AND 20 THEN 0.5 ELSE 0.25 END AS COEFICIENTE FROM FACT_SUBSCRIPCION
),

FACTURACION AS (
SELECT DISTINCT
    V.CLIENTE_ID,
    V.MES_ID,
    S.COEFICIENTE AS COEFICIENTE_COSTE_ENTREGA,
    S.COMPROMISO AS SUSCRIPCION,
    SUM(CANTIDAD) OVER (PARTITION BY V.CLIENTE_ID,V.MES_ID) - S.COMPROMISO AS EXTRA,
    SUM(CANTIDAD) OVER (PARTITION BY V.CLIENTE_ID,V.MES_ID) AS TOTAL,
    S.COMPROMISO * 9 AS INGRESOS_SUSCRIPCION_MES,
    S.COMPROMISO * S.COEFICIENTE * 1.5 AS COSTE_ENTREGA_SUB_MES,
    S.COMPROMISO * AVG(V.COSTE) OVER (PARTITION BY V.CLIENTE_ID,V.MES_ID) AS COSTE_SUSCRIPCION_MES,
    CASE WHEN S.COMPROMISO = 0 THEN (SUM(CANTIDAD) OVER (PARTITION BY V.CLIENTE_ID,V.MES_ID)) * 16 ELSE (SUM(CANTIDAD) OVER (PARTITION BY V.CLIENTE_ID,V.MES_ID) - S.COMPROMISO) * 12 END AS INGRESOS_EXTRA_MES,
    (SUM(CANTIDAD) OVER (PARTITION BY V.CLIENTE_ID,V.MES_ID) - S.COMPROMISO) * S.COEFICIENTE * 1.5 AS COSTE_ENTREGA_EXTRA_MES,
    (SUM(CANTIDAD) OVER (PARTITION BY V.CLIENTE_ID,V.MES_ID) - S.COMPROMISO) * AVG(V.COSTE) OVER (PARTITION BY V.CLIENTE_ID,V.MES_ID) AS COSTE_EXTRA_MES,
    CASE WHEN S.COMPROMISO = 0 THEN (SUM(CANTIDAD) OVER (PARTITION BY V.CLIENTE_ID,V.MES_ID)) * 16 ELSE (S.COMPROMISO * 9) + (SUM(CANTIDAD) OVER (PARTITION BY V.CLIENTE_ID,V.MES_ID) - S.COMPROMISO) * 12 END AS INGRESOS_TOTALES_MES,
    SUM(CANTIDAD) OVER (PARTITION BY V.CLIENTE_ID,V.MES_ID) * S.COEFICIENTE * 1.5 AS COSTE_ENTREGA_MES,
    (SUM(CANTIDAD) OVER (PARTITION BY V.CLIENTE_ID,V.MES_ID) * AVG(V.COSTE) OVER (PARTITION BY V.CLIENTE_ID,V.MES_ID)) AS COSTE_TOTAL_MES
FROM VENTA_COSTE V
LEFT JOIN SUSCRIPCION S 
    ON V.CLIENTE_ID = S.CLIENTE_ID AND S.MES_ID = V.MES_ID
),

DATOS_ANUALES AS (
        SELECT SUM(INGRESOS_SUSCRIPCION_MES) AS INGRESOS_SUSCRIPCION,
        SUM(COSTE_SUSCRIPCION_MES+COSTE_ENTREGA_SUB_MES) AS COSTE_SUSCRIPCION,
        (SUM(INGRESOS_SUSCRIPCION_MES) - SUM(COSTE_SUSCRIPCION_MES+COSTE_ENTREGA_SUB_MES))/SUM(INGRESOS_SUSCRIPCION_MES) AS MARGENBRUTO_SUSCRIPCION,
        SUM(INGRESOS_EXTRA_MES) AS INGRESOS_EXTRA,
        SUM(COSTE_EXTRA_MES+COSTE_ENTREGA_EXTRA_MES) AS COSTE_EXTRA,
        (SUM(INGRESOS_EXTRA_MES) - SUM(COSTE_EXTRA_MES+COSTE_ENTREGA_EXTRA_MES))/SUM(INGRESOS_EXTRA_MES) AS MARGENBRUTO_EXTRA,
        SUM(INGRESOS_TOTALES_MES) AS INGRESOS_TOTALES,
        SUM(COSTE_TOTAL_MES+COSTE_ENTREGA_MES) AS COSTES_TOTALES_BRUTO,
        SUM(COSTE_ENTREGA_MES) AS COSTE_ENTREGA,
        @COSTE_NOMINA*12 + @COSTE_RENTING*6*12 AS COSTE_PERSONAL,
        SUM(COSTE_TOTAL_MES+COSTE_ENTREGA_MES) + @COSTE_RENTING*6*12 + @COSTE_NOMINA*12 AS COSTES_TOTALES,
        (SUM(INGRESOS_TOTALES_MES) - SUM(COSTE_TOTAL_MES))/SUM(INGRESOS_TOTALES_MES) AS MARGEN_BRUTO,
        (SUM(INGRESOS_TOTALES_MES) - (@COSTE_NOMINA*12 + @COSTE_RENTING*6*12 + SUM(COSTE_TOTAL_MES+COSTE_ENTREGA_MES))) AS RESULTADO_OPERATIVO,
        (SUM(INGRESOS_TOTALES_MES) - (@COSTE_NOMINA*12 + @COSTE_RENTING*6*12 + SUM(COSTE_TOTAL_MES+COSTE_ENTREGA_MES)))/SUM(INGRESOS_TOTALES_MES) AS MARGEN_NETO
FROM FACTURACION
)

SELECT 
    CAST(COSTE_ENTREGA*100 / COSTES_TOTALES AS DECIMAL(5,2)) AS PCT_COSTE_ENTREGA,
    CAST(COSTE_PERSONAL * 100 / COSTES_TOTALES AS DECIMAL(5,2)) AS PCT_COSTE_PERSONAL,
    CAST((COSTES_TOTALES_BRUTO - COSTE_ENTREGA) * 100 / COSTES_TOTALES AS DECIMAL(5,2)) AS PCT_COSTE_PRODUCTO
FROM DATOS_ANUALES;

WITH VENTA_COSTE AS (
    SELECT V.CLIENTE_ID,V.PRODUCTO_ID,V.MES_ID,V.CANTIDAD,P.COSTE 
    FROM FACT_VENTA V
LEFT JOIN DIM_PRODUCTO P 
    ON V.PRODUCTO_ID = P.PRODUCTO_ID
),

SUSCRIPCION AS (
    SELECT *, CASE WHEN COMPROMISO <=10 THEN 1 WHEN COMPROMISO BETWEEN 10 AND 20 THEN 0.5 ELSE 0.25 END AS COEFICIENTE 
    FROM FACT_SUBSCRIPCION
),

DATOS_ANUALES AS (
SELECT DISTINCT
    V.CLIENTE_ID,
    V.MES_ID,
    S.COEFICIENTE AS COEFICIENTE_COSTE_ENTREGA,
    S.COMPROMISO AS SUSCRIPCION,
    SUM(CANTIDAD) OVER (PARTITION BY V.CLIENTE_ID,V.MES_ID) - S.COMPROMISO AS EXTRA,
    SUM(CANTIDAD) OVER (PARTITION BY V.CLIENTE_ID,V.MES_ID) AS TOTAL,
    S.COMPROMISO * 9 AS INGRESOS_SUSCRIPCION_MES,
    S.COMPROMISO * S.COEFICIENTE * 1.5 AS COSTE_ENTREGA_SUB_MES,
    S.COMPROMISO * AVG(V.COSTE) OVER (PARTITION BY V.CLIENTE_ID,V.MES_ID) AS COSTE_SUSCRIPCION_MES,
    CASE WHEN S.COMPROMISO = 0 THEN (SUM(CANTIDAD) OVER (PARTITION BY V.CLIENTE_ID,V.MES_ID)) * 16 ELSE (SUM(CANTIDAD) OVER (PARTITION BY V.CLIENTE_ID,V.MES_ID) - S.COMPROMISO) * 12 END AS INGRESOS_EXTRA_MES,
    (SUM(CANTIDAD) OVER (PARTITION BY V.CLIENTE_ID,V.MES_ID) - S.COMPROMISO) * S.COEFICIENTE * 1.5 AS COSTE_ENTREGA_EXTRA_MES,
    (SUM(CANTIDAD) OVER (PARTITION BY V.CLIENTE_ID,V.MES_ID) - S.COMPROMISO) * AVG(V.COSTE) OVER (PARTITION BY V.CLIENTE_ID,V.MES_ID) AS COSTE_EXTRA_MES,
    CASE WHEN S.COMPROMISO = 0 THEN (SUM(CANTIDAD) OVER (PARTITION BY V.CLIENTE_ID,V.MES_ID)) * 16 ELSE (S.COMPROMISO * 9) + (SUM(CANTIDAD) OVER (PARTITION BY V.CLIENTE_ID,V.MES_ID) - S.COMPROMISO) * 12 END AS INGRESOS_TOTALES_MES,
    SUM(CANTIDAD) OVER (PARTITION BY V.CLIENTE_ID,V.MES_ID) * S.COEFICIENTE * 1.5 AS COSTE_ENTREGA_MES,
    (SUM(CANTIDAD) OVER (PARTITION BY V.CLIENTE_ID,V.MES_ID) * AVG(V.COSTE) OVER (PARTITION BY V.CLIENTE_ID,V.MES_ID)) AS COSTE_TOTAL_MES
FROM VENTA_COSTE V
LEFT JOIN SUSCRIPCION S 
    ON V.CLIENTE_ID = S.CLIENTE_ID AND S.MES_ID = V.MES_ID
)

SELECT DISTINCT MES_ID, SUM(INGRESOS_TOTALES_MES) OVER (PARTITION BY MES_ID) AS INGRESOS_MES
FROM DATOS_ANUALES 
ORDER BY MES_ID ASC

(1 row affected)

(1 row affected)

(12 rows affected)

Total execution time: 00:00:01.003

INGRESOS_SUSCRIPCION,COSTE_SUSCRIPCION,MARGENBRUTO_SUSCRIPCION,INGRESOS_EXTRA,COSTE_EXTRA,MARGENBRUTO_EXTRA,INGRESOS_TOTALES,COSTES_TOTALES_BRUTO,COSTE_ENTREGA,COSTE_PERSONAL,COSTES_TOTALES,MARGEN_BRUTO,RESULTADO_OPERATIVORESULTADO_OPERATIVO,MARGEN_NETO
271071,"192454,8267701741","0,29002059692783766",66828,"35100,332610722675","0,47476607693298206",337899,"227555,15938089698","34680,750",126600,"354155,15938089695","0,4291950867540396","-16256,159380896948","-0,04810952201958854"


PCT_COSTE_ENTREGA,PCT_COSTE_PERSONAL,PCT_COSTE_PRODUCTO
"9,79","35,75","54,46"


MES_ID,INGRESOS_MES
201905,7509
201906,12477
201907,17958
201908,22212
201909,17370
201910,18612
201911,21321
201912,23841
202001,28971
202002,32607


Observando los datos, es sencillo detectar que la empresa ha tenido pérdidas a lo largo de los doce últimos meses. Sin embargo, y pese a que la estrategia global es más que mejorable y optimizable, no se puede categorizar como un fracaso. En líneas generales, los ingresos han ido al alza, y la consulta de abajo muestra que, si sólo contabilizamos los meses de 2020, la empresa tendría resultado positivo. Estos números pueden considerarse aceptables teniendo en cuenta que los datos muestran que la empresa arrancó desde cero con el personal que tiene actualmente y que sólo ha pasado un año desde el inicio de la actividad. 

**Así, asumiendo que el mantenimiento de la cartera de clientes seguiría como hasta ahora, es sencillo inferir que la empresa tendría beneficios** **sostenidos y un margen neto aceptable a partir de su segundo año de actividad, pese a la mejorable gestión en materia de costes y estrategia comercial. Como veremos abajo, las medidas que se proponen inciden en esos dos puntos, pues se considera que es ahí donde existe mayor margen para optimizar.**

In [7]:
DECLARE @COSTE_NOMINA AS NUMERIC
SET @COSTE_NOMINA = (SELECT DISTINCT SUM(IMPORTE_NOMINA) OVER (PARTITION BY MES_ID) FROM FACT_NOMINA)
DECLARE @COSTE_RENTING AS INT = 250;

WITH VENTA_COSTE AS (
    SELECT V.CLIENTE_ID,V.PRODUCTO_ID,V.MES_ID,V.CANTIDAD,P.COSTE FROM FACT_VENTA V
LEFT JOIN DIM_PRODUCTO P 
    ON V.PRODUCTO_ID = P.PRODUCTO_ID
),

SUSCRIPCION AS (
    SELECT *, CASE WHEN COMPROMISO <=10 THEN 1 WHEN COMPROMISO BETWEEN 10 AND 20 THEN 0.5 ELSE 0.25 END AS COEFICIENTE FROM FACT_SUBSCRIPCION
),

FACTURACION AS (
SELECT DISTINCT
    V.CLIENTE_ID,
    V.MES_ID,
    S.COEFICIENTE AS COEFICIENTE_COSTE_ENTREGA,
    S.COMPROMISO AS SUSCRIPCION,
    SUM(CANTIDAD) OVER (PARTITION BY V.CLIENTE_ID,V.MES_ID) - S.COMPROMISO AS EXTRA,
    SUM(CANTIDAD) OVER (PARTITION BY V.CLIENTE_ID,V.MES_ID) AS TOTAL,
    S.COMPROMISO * 9 AS INGRESOS_SUSCRIPCION_MES,
    S.COMPROMISO * S.COEFICIENTE * 1.5 AS COSTE_ENTREGA_SUB_MES,
    S.COMPROMISO * AVG(V.COSTE) OVER (PARTITION BY V.CLIENTE_ID,V.MES_ID) AS COSTE_SUSCRIPCION_MES,
    CASE WHEN S.COMPROMISO = 0 THEN (SUM(CANTIDAD) OVER (PARTITION BY V.CLIENTE_ID,V.MES_ID)) * 16 ELSE (SUM(CANTIDAD) OVER (PARTITION BY V.CLIENTE_ID,V.MES_ID) - S.COMPROMISO) * 12 END AS INGRESOS_EXTRA_MES,
    (SUM(CANTIDAD) OVER (PARTITION BY V.CLIENTE_ID,V.MES_ID) - S.COMPROMISO) * S.COEFICIENTE * 1.5 AS COSTE_ENTREGA_EXTRA_MES,
    (SUM(CANTIDAD) OVER (PARTITION BY V.CLIENTE_ID,V.MES_ID) - S.COMPROMISO) * AVG(V.COSTE) OVER (PARTITION BY V.CLIENTE_ID,V.MES_ID) AS COSTE_EXTRA_MES,
    CASE WHEN S.COMPROMISO = 0 THEN (SUM(CANTIDAD) OVER (PARTITION BY V.CLIENTE_ID,V.MES_ID)) * 16 ELSE (S.COMPROMISO * 9) + (SUM(CANTIDAD) OVER (PARTITION BY V.CLIENTE_ID,V.MES_ID) - S.COMPROMISO) * 12 END AS INGRESOS_TOTALES_MES,
    SUM(CANTIDAD) OVER (PARTITION BY V.CLIENTE_ID,V.MES_ID) * S.COEFICIENTE * 1.5 AS COSTE_ENTREGA_MES,
    (SUM(CANTIDAD) OVER (PARTITION BY V.CLIENTE_ID,V.MES_ID) * AVG(V.COSTE) OVER (PARTITION BY V.CLIENTE_ID,V.MES_ID)) AS COSTE_TOTAL_MES
FROM VENTA_COSTE V
LEFT JOIN SUSCRIPCION S 
    ON V.CLIENTE_ID = S.CLIENTE_ID AND S.MES_ID = V.MES_ID
)

SELECT SUM(INGRESOS_SUSCRIPCION_MES) AS INGRESOS_SUSCRIPCION,
        SUM(COSTE_SUSCRIPCION_MES+COSTE_ENTREGA_SUB_MES) AS COSTE_SUSCRIPCION,
        (SUM(INGRESOS_SUSCRIPCION_MES) - SUM(COSTE_SUSCRIPCION_MES+COSTE_ENTREGA_SUB_MES))/SUM(INGRESOS_SUSCRIPCION_MES) AS MARGENBRUTO_SUSCRIPCION,
        SUM(INGRESOS_EXTRA_MES) AS INGRESOS_EXTRA,
        SUM(COSTE_EXTRA_MES+COSTE_ENTREGA_EXTRA_MES) AS COSTE_EXTRA,
        (SUM(INGRESOS_EXTRA_MES) - SUM(COSTE_EXTRA_MES+COSTE_ENTREGA_EXTRA_MES))/SUM(INGRESOS_EXTRA_MES) AS MARGENBRUTO_EXTRA,
        SUM(INGRESOS_TOTALES_MES) AS INGRESOS_TOTALES,
        SUM(COSTE_TOTAL_MES+COSTE_ENTREGA_MES) AS COSTES_TOTALES_BRUTO,
        SUM(COSTE_ENTREGA_MES) AS COSTE_ENTREGA,
        @COSTE_NOMINA*4 + @COSTE_RENTING*6*4 AS COSTE_PERSONAL,
        SUM(COSTE_TOTAL_MES+COSTE_ENTREGA_MES) + @COSTE_RENTING*6*4 + @COSTE_NOMINA*4 AS COSTES_TOTALES,
        (SUM(INGRESOS_TOTALES_MES) - SUM(COSTE_TOTAL_MES))/SUM(INGRESOS_TOTALES_MES) AS MARGEN_BRUTO,
        (SUM(INGRESOS_TOTALES_MES) - (@COSTE_NOMINA*4 + @COSTE_RENTING*6*4 + SUM(COSTE_TOTAL_MES+COSTE_ENTREGA_MES))) AS RESULTADO_OPERATIVO,
        (SUM(INGRESOS_TOTALES_MES) - (@COSTE_NOMINA*4 + @COSTE_RENTING*6*4 + SUM(COSTE_TOTAL_MES+COSTE_ENTREGA_MES)))/SUM(INGRESOS_TOTALES_MES) AS MARGEN_NETO
FROM FACTURACION
WHERE MES_ID >=202000

(1 row affected)

Total execution time: 00:00:00.282

INGRESOS_SUSCRIPCION,COSTE_SUSCRIPCION,MARGENBRUTO_SUSCRIPCION,INGRESOS_EXTRA,COSTE_EXTRA,MARGENBRUTO_EXTRA,INGRESOS_TOTALES,COSTES_TOTALES_BRUTO,COSTE_ENTREGA,COSTE_PERSONAL,COSTES_TOTALES,MARGEN_BRUTO,RESULTADO_OPERATIVO,MARGEN_NETO
149967,"106093,41443556447","0,2925549325147235",46632,"24491,308354423374","0,4747960980780714",196599,"130584,72278998769","19481,250",42200,"172784,7227899877","0,4348726453848305","23814,27721001231","0,12113122248847813"


Se asume que el gasto de personal es excesivo para el área que se trata de abarcar en la operativa empresarial. Existe un cocinero en nómina, que debe preparar un total de aprox. 5000 ensaladas mensuales, y seis comerciales que deben gestionar una red de, actualmente, 404 clientes suscritos en seis zonas geográficas distintas. Sabemos también que el área de influencia que se pretende tener es equivalente (aprox.) a 2/3 de la extensión total de Galicia (-**VER "AREA\_TOTAL.PNG" en ANEXOS**\-), lo que nos impulsa a creer que la operativa comercial puede simplificarse. Además, estos efectivos, desde nuestro punto de vista, están ahora destinados a realizar funciones con poco retorno -viajar a mostrar in situ cada nuevo producto a los clientes actuales-.

Para preservar la viabilidad económica del negocio, acciones como las descritas aquí y en el enunciado pueden sustituirse por una gestión centralizada que venga dada por el tratamiento y el análisis de los datos de los que disponemos y dispondremos. Con el modelo de gestión de clientes adecuado, es posible identificar patrones que nos permitan saber qué parte de nuestra cartera de clientes debe ser tratada con una especial atención en un momento dado. Asimismo, la actuación a realizar, llegado el punto, no tiene por qué ser presencial, ya que actualmente contamos con sistemas de fidelización y captación que se pueden ejecutar sin la necesidad de una mediación comercial (newsletters, promociones, estrategias de posicionamiento en redes, etc). Obviamente, existen casos en los que la presencialidad y la atención personalizada seguirán siendo necesarias, y, por ello, consideramos que no se puede prescindir de la red comercial, si bien, como hemos dicho, la operativa debe ser optimizada y simplificada de forma obligatoria.

Para poder valorar en su justa medida la estrategia comercial seguida debemos analizar la composición de nuestra cartera de clientes, y ver cómo ésta ha evolucionado en el plano temporal y a nivel geográfico. Podremos establecer, así, si existe un margen de evolución a nivel de ventas, o si sólo podemos ofrecer una solución a nivel de reducción de costes.

## **PREPARACIÓN ESTRATEGIA (I) -** **Análisis Cartera Clientes**

In [8]:
WITH AGRUPADO1 AS (SELECT MES_ID,CLIENTE_ID,COMPROMISO FROM FACT_SUBSCRIPCION)
SELECT COUNT(DISTINCT CLIENTE_ID) AS NUM_CLIENTES_SUSCRITOS_DISTINTOS FROM AGRUPADO1;

WITH AGRUPADO1 AS (SELECT MES_ID,CLIENTE_ID,COMPROMISO FROM FACT_SUBSCRIPCION)
SELECT DISTINCT COUNT(*) OVER (PARTITION BY MES_ID) AS NUM_CLIENTES_DISTINTOS_SUSCRITOS_ABRIL2020 FROM AGRUPADO1 WHERE MES_ID = 202004;

WITH AGRUPADO2 AS (
SELECT DISTINCT MES_ID,CLIENTE_ID FROM FACT_VENTA
)

SELECT COUNT(DISTINCT CLIENTE_ID) AS NUM_CLIENTES_DISTINTOS FROM AGRUPADO2;

WITH AGRUPADO2 AS (
SELECT DISTINCT MES_ID,CLIENTE_ID FROM FACT_VENTA
)

SELECT DISTINCT COUNT(*) OVER (PARTITION BY MES_ID) AS NUM_CLIENTES_DISTINTOS_ABRIL2020 FROM AGRUPADO2 WHERE MES_ID = 202004;

(1 row affected)

(1 row affected)

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.117

NUM_CLIENTES_SUSCRITOS_DISTINTOS
404


NUM_CLIENTES_DISTINTOS_SUSCRITOS_ABRIL2020
404


NUM_CLIENTES_DISTINTOS
404


NUM_CLIENTES_DISTINTOS_ABRIL2020
404


In [9]:
WITH SUSCRIPCIONES AS (
SELECT DISTINCT MES_ID,
                CAST(COUNT(CLIENTE_ID) OVER (PARTITION BY MES_ID) AS DECIMAL(5,2)) AS CLIENTES_SUSCRITOS,
                SUM(COMPROMISO) OVER (PARTITION BY MES_ID) AS ENSALADAS
FROM FACT_SUBSCRIPCION
)

SELECT MES_ID, 
       CLIENTES_SUSCRITOS,
       ENSALADAS,
       CAST((CLIENTES_SUSCRITOS - LAG(CLIENTES_SUSCRITOS,1) OVER (ORDER BY MES_ID)) / (LAG(CLIENTES_SUSCRITOS,1) OVER (ORDER BY MES_ID)) *100 AS DECIMAL(5,2)) AS INCREMENTO_SUBS
FROM SUSCRIPCIONES

(12 rows affected)

Total execution time: 00:00:00.119

MES_ID,CLIENTES_SUSCRITOS,ENSALADAS,INCREMENTO_SUBS
201905,"41,00",557,NULL
201906,"76,00",921,"85,37"
201907,"108,00",1330,"42,11"
201908,"132,00",1632,"22,22"
201909,"173,00",1930,"31,06"
201910,"201,00",2068,"16,18"
201911,"235,00",2369,"16,92"
201912,"269,00",2649,"14,47"
202001,"306,00",3219,"13,75"
202002,"339,00",3623,"10,78"


Se demuestra, mediante las consultas anteriores, que el nº de clientes suscritos distintos a lo largo del tiempo es igual que el nº de clientes que alguna vez han hecho compra (404), y que dicho número es igual a lo que se ha registrado, tanto en clientes como en suscripciones, en abril de 2020. Por tanto, sabemos que:

>  · Todos los clientes que han hecho compras fuera de suscripción estaban suscritos (no hay ningún cliente que haya hecho una compra sin estar suscrito).
> 
>  · Todos los clientes estaban suscritos en abril de 2020. Por tanto, no se ha perdido ningún cliente desde el inicio de la actividad. En un RFM, todos los clientes tendrán el máximo en recencia.
> 
>  ·  La captación de clientes nuevos tiende a estabilizarse con el tiempo.

Para saber si cabe actuar en lo referente al modelo de captación de clientes de los últimos doce meses debemos estudiar cómo se distribuyen éstos por zonas y, a su vez, en qué zonas se ha centralizado la estrategia comercial.

In [10]:
SELECT COUNT (DISTINCT CLIENTE_ID) AS CLIENTES_DISTINTOS_ULTIMO_MES FROM FACT_SUBSCRIPCION WHERE MES_ID = 202004;

SELECT * FROM DIM_ZONA;

SELECT DISTINCT CAST(LATITUD_ID AS INT) AS LATITUD_CLIENTE, CAST(LONGITUD_ID AS INT) AS LONGITUD_CLIENTE FROM  DIM_CLIENTE;

(1 row affected)

(6 rows affected)

(7 rows affected)

Total execution time: 00:00:00.112

CLIENTES_DISTINTOS_ULTIMO_MES
404


ZONA_ID,LATITUD_GRADO,LONGITUD_GRADO
ZONA - 1,31,-9
ZONA - 2,32,-9
ZONA - 3,32,-8
ZONA - 4,33,-9
ZONA - 5,33,-8
ZONA - 6,34,-8


LATITUD_CLIENTE,LONGITUD_CLIENTE
30,-9
31,-9
31,-8
32,-9
32,-8
33,-8
33,-7


Con la simple consulta previa podemos ver que hay zonas, dentro de nuestro área de influencia (entiéndase tal como las seis zonas de la tabla DIM\_ZONA), en las que no se registra ni un sólo cliente dado de alta en la base de datos (tabla DIM\_CLIENTE). Entendemos que hay margen de mejora a nivel geográfico, por lo que profundizamos el análisis en ese sentido.

In [11]:
WITH SUSCRIPCION AS (
SELECT DISTINCT 
    S.CLIENTE_ID, 
    SUM(S.COMPROMISO) OVER (PARTITION BY S.CLIENTE_ID) AS COMPROMISO, 
    CAST(C.LATITUD_ID AS INT) AS LATITUD, 
    CAST(C.LONGITUD_ID AS INT) AS LONGITUD,
    CASE WHEN CAST(C.LATITUD_ID AS INT)=33 THEN 'ZONA - 5' WHEN CAST(C.LATITUD_ID AS INT)=31 AND CAST(C.LONGITUD_ID AS INT)=-8 THEN 'ZONA - 1' WHEN CAST(C.LATITUD_ID AS INT)=30 THEN 'ZONA - 1' ELSE Z.ZONA_ID END AS ZONA_ID
FROM FACT_SUBSCRIPCION S
LEFT JOIN DIM_CLIENTE C 
    ON C.CLIENTE_ID = S.CLIENTE_ID
LEFT JOIN DIM_ZONA Z 
    ON Z.LATITUD_GRADO = CAST(C.LATITUD_ID AS INT) AND Z.LONGITUD_GRADO = CAST(C.LONGITUD_ID AS INT)
WHERE S.MES_ID >= 202001
),

POSICIONAMIENTO AS (
    SELECT  TS_POSICION,
            POSICIONABLE_ID,
            POSICIONAMIENTO_ESTADO,
            LAG(POSICIONAMIENTO_ESTADO) OVER (PARTITION BY POSICIONABLE_ID ORDER BY TS_POSICION) AS ESTADO_ANTERIOR,
            LEAD(TS_POSICION) OVER (PARTITION BY POSICIONABLE_ID ORDER BY TS_POSICION ASC) AS SIGUIENTE_ESTADO,
            CASE WHEN POSICIONAMIENTO_ESTADO <> LAG(POSICIONAMIENTO_ESTADO) OVER(PARTITION BY POSICIONABLE_ID ORDER BY TS_POSICION) AND POSICIONAMIENTO_ESTADO = 'Encendido' THEN 'ARRANQUE' ELSE 'PARADA' END AS ESTADO_VEHICULO,
            LAG(LATITUD_ID) OVER (PARTITION BY POSICIONABLE_ID ORDER BY TS_POSICION) AS LAT_ANTERIOR,
            LAG(LONGITUD_ID) OVER (PARTITION BY POSICIONABLE_ID ORDER BY TS_POSICION) AS LON_ANTERIOR,
            CASE WHEN (POSICIONAMIENTO_ESTADO <> LAG(POSICIONAMIENTO_ESTADO) OVER (PARTITION BY POSICIONABLE_ID ORDER BY TS_POSICION)) THEN 1 ELSE 0 END AS CAMBIO,
            POSICION_VELOCIDAD, 
            LATITUD_ID, 
            LONGITUD_ID
FROM NSLDINA.DBO.FACT_POSICION
),

POSICION AS (
    SELECT * FROM POSICIONAMIENTO 
    WHERE DATEDIFF(SECOND, TS_POSICION, SIGUIENTE_ESTADO) >= 60
),

COMERCIAL_POS AS (
    SELECT DISTINCT POSICIONABLE_ID,
                CAST(AVG(LATITUD_ID) OVER (PARTITION BY POSICIONABLE_ID) AS INT) AS LAT, 
                CAST(AVG(LONGITUD_ID) OVER (PARTITION BY POSICIONABLE_ID) AS INT) AS LON
FROM POSICION
),

COMERCIAL_ZONA AS (
    SELECT C.*,Z.ZONA_ID, COUNT(*) OVER (PARTITION BY Z.ZONA_ID) AS COMERCIALES
    FROM COMERCIAL_POS C 
    LEFT JOIN DIM_ZONA Z 
        ON Z.LATITUD_GRADO = C.LAT AND Z.LONGITUD_GRADO = C.LON
),

SUSCRIPCION_ZONA AS(
SELECT DISTINCT Z.ZONA_ID,
                Z.LATITUD_GRADO AS LAT,
                Z.LONGITUD_GRADO AS LON,
                COUNT(S.CLIENTE_ID) OVER (PARTITION BY Z.ZONA_ID) AS N_CLIENTES_FIDELIZADOS,
                CASE WHEN SUM(COMPROMISO) OVER (PARTITION BY S.ZONA_ID) IS NULL THEN 0 ELSE SUM(COMPROMISO) OVER (PARTITION BY S.ZONA_ID) END AS TOTAL_ENSALADAS_2020,
                CASE WHEN (SUM(COMPROMISO) OVER (PARTITION BY S.ZONA_ID) / COUNT(*) OVER (PARTITION BY S.ZONA_ID))/4 IS NULL THEN 0 ELSE (SUM(COMPROMISO) OVER (PARTITION BY S.ZONA_ID)/4) END AS ENSALADAS_MES,
                CASE WHEN (SUM(COMPROMISO) OVER (PARTITION BY S.ZONA_ID) / COUNT(*) OVER (PARTITION BY S.ZONA_ID))/4 IS NULL THEN 0 ELSE (SUM(COMPROMISO) OVER (PARTITION BY S.ZONA_ID) / COUNT(*) OVER (PARTITION BY S.ZONA_ID))/4 END AS ENSALADAS_CLIENTE_MES
FROM DIM_ZONA Z 
LEFT JOIN SUSCRIPCION S 
    ON Z.ZONA_ID = S.ZONA_ID
)

SELECT DISTINCT S.*,CASE WHEN C.COMERCIALES IS NULL THEN 0 ELSE C.COMERCIALES END AS N_COMERCIALES_ZONA 
FROM SUSCRIPCION_ZONA S 
LEFT JOIN COMERCIAL_ZONA C 
    ON C.ZONA_ID = S.ZONA_ID
ORDER BY S.ZONA_ID ASC;


WITH POSICIONAMIENTO AS (
    SELECT  TS_POSICION,
            POSICIONABLE_ID,
            POSICIONAMIENTO_ESTADO,
            LAG(POSICIONAMIENTO_ESTADO) OVER (PARTITION BY POSICIONABLE_ID ORDER BY TS_POSICION) AS ESTADO_ANTERIOR,
            LEAD(TS_POSICION) OVER (PARTITION BY POSICIONABLE_ID ORDER BY TS_POSICION ASC) AS SIGUIENTE_ESTADO,
            CASE WHEN POSICIONAMIENTO_ESTADO <> LAG(POSICIONAMIENTO_ESTADO) OVER(PARTITION BY POSICIONABLE_ID ORDER BY TS_POSICION) AND POSICIONAMIENTO_ESTADO = 'Encendido' THEN 'ARRANQUE' ELSE 'PARADA' END AS ESTADO_VEHICULO,
            LAG(LATITUD_ID) OVER (PARTITION BY POSICIONABLE_ID ORDER BY TS_POSICION) AS LAT_ANTERIOR,
            LAG(LONGITUD_ID) OVER (PARTITION BY POSICIONABLE_ID ORDER BY TS_POSICION) AS LON_ANTERIOR,
            CASE WHEN (POSICIONAMIENTO_ESTADO <> LAG(POSICIONAMIENTO_ESTADO) OVER (PARTITION BY POSICIONABLE_ID ORDER BY TS_POSICION)) THEN 1 ELSE 0 END AS CAMBIO,
            POSICION_VELOCIDAD, 
            LATITUD_ID, 
            LONGITUD_ID
FROM NSLDINA.DBO.FACT_POSICION
),

POSICION AS (
    SELECT * FROM POSICIONAMIENTO 
    WHERE DATEDIFF(SECOND, TS_POSICION, SIGUIENTE_ESTADO) >= 60
),

COMERCIAL_POS AS (
    SELECT DISTINCT POSICIONABLE_ID,
                CAST(AVG(LATITUD_ID) OVER (PARTITION BY POSICIONABLE_ID) AS INT) AS LAT, 
                CAST(AVG(LONGITUD_ID) OVER (PARTITION BY POSICIONABLE_ID) AS INT) AS LON
FROM POSICION
),

COMERCIAL_ZONA AS (
    SELECT C.*,Z.ZONA_ID, COUNT(*) OVER (PARTITION BY Z.ZONA_ID) AS COMERCIALES
    FROM COMERCIAL_POS C 
    LEFT JOIN DIM_ZONA Z 
        ON Z.LATITUD_GRADO = C.LAT AND Z.LONGITUD_GRADO = C.LON
)

SELECT POSICIONABLE_ID AS COMERCIAL,ZONA_ID AS LOCALIZACION_MEDIA_COMERCIAL,COMERCIALES AS N_COMERCIALES_ZONA FROM COMERCIAL_ZONA;

Advertencia: valor NULL eliminado por el agregado u otra operación SET.

(6 rows affected)

Advertencia: valor NULL eliminado por el agregado u otra operación SET.

(6 rows affected)

Total execution time: 00:00:00.653

ZONA_ID,LAT,LON,N_CLIENTES_FIDELIZADOS,TOTAL_ENSALADAS_2020,ENSALADAS_MES,ENSALADAS_CLIENTE_MES,N_COMERCIALES_ZONA
ZONA - 1,31,-9,160,6564,1641,10,2
ZONA - 2,32,-9,90,3732,933,10,1
ZONA - 3,32,-8,145,6038,1509,10,3
ZONA - 4,33,-9,0,0,0,0,0
ZONA - 5,33,-8,9,329,82,9,0
ZONA - 6,34,-8,0,0,0,0,0


COMERCIAL,LOCALIZACION_MEDIA_COMERCIAL,N_COMERCIALES_ZONA
129,ZONA - 1,2
121,ZONA - 1,2
135,ZONA - 2,1
137,ZONA - 3,3
115,ZONA - 3,3
116,ZONA - 3,3


De los 404 clientes fidelizados (con suscripciones ininterrumpidas durante todos los meses de 2020), 395 se encuentran en las zonas que, observando el mapa, se pueden considerar "sur" (zonas 1, 2 y 3) , mientras que en la zona "norte" (4, 5 y 6)  se han vendido un total de 329 ensaladas (por suscripción) en 2020 y sólo se registran 9 suscripciones. Ello nos impulsa a pensar que tenemos sin abordar comercialmente un área de al menos 10.000 km2 (1/3 de las dimensiones de Galicia -**VER "ZONAS\_4\_5\_6.PNG" en ANEXOS**\-) que, si bien se encontraba en nuestras previsiones de explotación (se encuentra en la tabla DIM\_ZONA), no se está trabajando. De hecho, si observamos la tabla FACT\_POSICION, veremos que las posiciones medias de todos los comerciales se sitúan en las zonas ya "explotadas" (las zonas 1, 2 y 3, donde se han añadido la gran mayoría de clientes hasta hoy).

En las consultas previas también hemos visto que las altas de clientes (que como hemos dicho se concentran siempre en las mismas zonas) han tendido, como es lógico, a normalizarse. En concreto, es previsible que, de mantener la estrategia comercial en las mismas zonas que hasta ahora, las altas, de crecer, lo harían a una tasa nunca superior al 10%. Por ello, y sabiendo que se encuentra sin explotar comercialmente la mitad de nuestro área de influencia, se recomienda no sólo limitar la operativa de los comerciales a la consecución de nuevos clientes -salvo las excepciones mencionadas- sino, además, centralizarla en las zonas 4, 5 y 6 que, como hemos dicho, apenas han sido explotadas.

## **PREPARACIÓN ESTRATEGIA (II) -** **Análisis Rendimiento Red Comercial**

A lo largo de este proyecto se comentan diversos puntos a optimizar en lo referente a la estrategia comercial. Uno de ellos hace referencia al exceso de comerciales en nómina teniendo en cuenta la extensión del área de influencia total de la empresa (que se estima en unos 20.000 km. cuadrados -suma de la extensión de cada área de la tabla DIM\_ZONA-). Por ello, entendemos que es necesario realizar un análisis del rendimiento de los comerciales a lo largo del año del cuál tenemos datos, de cara a entender quiénes han aportado mayor valor y de quién/es se podría prescindir con un menor coste de oportunidad.

  

De este modo, se procede, partiendo de los datos de la tabla FACT\_POSICION, a identificar a cada cliente con su comercial. Tras ello, extraeremos, empleando la tabla FACTURACION usada previamente en el proyecto, los ingresos totales que son atribuibles a los clientes de cada comercial y, finalmente, el retorno por cada euro que cuesta a la empresa el mantener a cada comercial (teniendo en cuenta el coste mensual de los rentings de vehículos).

In [12]:
WITH POSICIONAMIENTO AS (
    SELECT  TS_POSICION,
            POSICIONABLE_ID,
            POSICIONAMIENTO_ESTADO,
            LAG(POSICIONAMIENTO_ESTADO) OVER (PARTITION BY POSICIONABLE_ID ORDER BY TS_POSICION) AS ESTADO_ANTERIOR,
            LEAD(TS_POSICION) OVER (PARTITION BY POSICIONABLE_ID ORDER BY TS_POSICION ASC) AS SIGUIENTE_ESTADO,
            CASE WHEN POSICIONAMIENTO_ESTADO <> LAG(POSICIONAMIENTO_ESTADO) OVER(PARTITION BY POSICIONABLE_ID ORDER BY TS_POSICION) AND POSICIONAMIENTO_ESTADO = 'Encendido' THEN 'ARRANQUE' ELSE 'PARADA' END AS ESTADO_VEHICULO,
            LAG(LATITUD_ID) OVER (PARTITION BY POSICIONABLE_ID ORDER BY TS_POSICION) AS LAT_ANTERIOR,
            LAG(LONGITUD_ID) OVER (PARTITION BY POSICIONABLE_ID ORDER BY TS_POSICION) AS LON_ANTERIOR,
            CASE WHEN (POSICIONAMIENTO_ESTADO <> LAG(POSICIONAMIENTO_ESTADO) OVER (PARTITION BY POSICIONABLE_ID ORDER BY TS_POSICION)) THEN 1 ELSE 0 END AS CAMBIO,
            POSICION_VELOCIDAD, 
            LATITUD_ID, 
            LONGITUD_ID
FROM FACT_POSICION
),

POSICION AS (
    SELECT * FROM POSICIONAMIENTO 
    WHERE DATEDIFF(SECOND, TS_POSICION, SIGUIENTE_ESTADO) >= 20
),

CLIENTES AS (
    SELECT * FROM DIM_CLIENTE
),

ASIGNACION AS (
SELECT P.POSICIONABLE_ID,
       CAST(P.LATITUD_ID AS DECIMAL (5,2)) AS LATITUD,
       CAST(P.LONGITUD_ID AS DECIMAL (5,2)) AS LONGITUD,
       C.CLIENTE_ID, 
       C.MES_ALTA_ID AS MES_ALTA_CLIENTE
FROM POSICION P
LEFT JOIN DIM_CLIENTE C 
    ON ABS(P.LATITUD_ID - C.LATITUD_ID) < 0.001
    AND ABS(P.LONGITUD_ID - C.LONGITUD_ID) < 0.001
WHERE C.CLIENTE_ID IS NOT NULL
),

CONTEO_LINEAS AS (
SELECT DISTINCT POSICIONABLE_ID,
                CLIENTE_ID,
                COUNT(*) OVER (PARTITION BY POSICIONABLE_ID,CLIENTE_ID) AS LINEAS_COMERCIAL_CLIENTE
                
FROM ASIGNACION
),

NUMERADO AS (
SELECT *,ROW_NUMBER() OVER(PARTITION BY CLIENTE_ID ORDER BY LINEAS_COMERCIAL_CLIENTE DESC) AS RN 
FROM CONTEO_LINEAS
)

SELECT TOP 10 CLIENTE_ID, POSICIONABLE_ID AS COMERCIAL FROM NUMERADO
WHERE RN=1

(10 rows affected)

Total execution time: 00:00:10.230

CLIENTE_ID,COMERCIAL
Cliente - 030,116
Cliente - 008,135
Cliente - 036,135
Cliente - 046,135
Cliente - 017,135
Cliente - 009,129
Cliente - 076,135
Cliente - 001,129
Cliente - 078,129
Cliente - 011,137


In [13]:
DECLARE @COSTE_RENTING AS INT = 250;
DECLARE @INGRESOS_TOTALES AS INT = 337899;

WITH POSICIONAMIENTO AS (
    SELECT  TS_POSICION,
            POSICIONABLE_ID,
            POSICIONAMIENTO_ESTADO,
            LAG(POSICIONAMIENTO_ESTADO) OVER (PARTITION BY POSICIONABLE_ID ORDER BY TS_POSICION) AS ESTADO_ANTERIOR,
            LEAD(TS_POSICION) OVER (PARTITION BY POSICIONABLE_ID ORDER BY TS_POSICION ASC) AS SIGUIENTE_ESTADO,
            CASE WHEN POSICIONAMIENTO_ESTADO <> LAG(POSICIONAMIENTO_ESTADO) OVER(PARTITION BY POSICIONABLE_ID ORDER BY TS_POSICION) AND POSICIONAMIENTO_ESTADO = 'Encendido' THEN 'ARRANQUE' ELSE 'PARADA' END AS ESTADO_VEHICULO,
            LAG(LATITUD_ID) OVER (PARTITION BY POSICIONABLE_ID ORDER BY TS_POSICION) AS LAT_ANTERIOR,
            LAG(LONGITUD_ID) OVER (PARTITION BY POSICIONABLE_ID ORDER BY TS_POSICION) AS LON_ANTERIOR,
            CASE WHEN (POSICIONAMIENTO_ESTADO <> LAG(POSICIONAMIENTO_ESTADO) OVER (PARTITION BY POSICIONABLE_ID ORDER BY TS_POSICION)) THEN 1 ELSE 0 END AS CAMBIO,
            POSICION_VELOCIDAD, 
            LATITUD_ID, 
            LONGITUD_ID
FROM FACT_POSICION
),

POSICION AS (
    SELECT * FROM POSICIONAMIENTO 
    WHERE DATEDIFF(SECOND, TS_POSICION, SIGUIENTE_ESTADO) >= 20
),

CLIENTES AS (
    SELECT * FROM DIM_CLIENTE
),

ASIGNACION AS (
SELECT P.POSICIONABLE_ID,
       CAST(P.LATITUD_ID AS DECIMAL (5,2)) AS LATITUD,
       CAST(P.LONGITUD_ID AS DECIMAL (5,2)) AS LONGITUD,
       C.CLIENTE_ID, 
       C.MES_ALTA_ID AS MES_ALTA_CLIENTE
FROM POSICION P
LEFT JOIN DIM_CLIENTE C 
    ON ABS(P.LATITUD_ID - C.LATITUD_ID) < 0.001
    AND ABS(P.LONGITUD_ID - C.LONGITUD_ID) < 0.001
WHERE C.CLIENTE_ID IS NOT NULL
),

CONTEO_LINEAS AS (
SELECT DISTINCT POSICIONABLE_ID,
                CLIENTE_ID,
                COUNT(*) OVER (PARTITION BY POSICIONABLE_ID,CLIENTE_ID) AS LINEAS_COMERCIAL_CLIENTE
                
FROM ASIGNACION
),

NUMERADO AS (
SELECT *,ROW_NUMBER() OVER(PARTITION BY CLIENTE_ID ORDER BY LINEAS_COMERCIAL_CLIENTE DESC) AS RN 
FROM CONTEO_LINEAS
),

COMERCIAL_CLIENTE AS (
    SELECT CLIENTE_ID, POSICIONABLE_ID AS COMERCIAL FROM NUMERADO
    WHERE RN=1
),

VENTA_COSTE AS (
    SELECT V.CLIENTE_ID,V.PRODUCTO_ID,V.MES_ID,V.CANTIDAD,P.COSTE 
    FROM FACT_VENTA V
LEFT JOIN DIM_PRODUCTO P 
    ON V.PRODUCTO_ID = P.PRODUCTO_ID
),

SUSCRIPCION AS (
    SELECT *, CASE WHEN COMPROMISO <=10 THEN 1 WHEN COMPROMISO BETWEEN 10 AND 20 THEN 0.5 ELSE 0.25 END AS COEFICIENTE 
    FROM FACT_SUBSCRIPCION
),

FACTURACION AS (
SELECT DISTINCT
    V.CLIENTE_ID,
    V.MES_ID,
    S.COEFICIENTE AS COEFICIENTE_COSTE_ENTREGA,
    S.COMPROMISO AS SUSCRIPCION,
    SUM(CANTIDAD) OVER (PARTITION BY V.CLIENTE_ID,V.MES_ID) - S.COMPROMISO AS EXTRA,
    SUM(CANTIDAD) OVER (PARTITION BY V.CLIENTE_ID,V.MES_ID) AS TOTAL,
    S.COMPROMISO * 9 AS INGRESOS_SUSCRIPCION_MES,
    S.COMPROMISO * S.COEFICIENTE * 1.5 AS COSTE_ENTREGA_SUB_MES,
    S.COMPROMISO * AVG(V.COSTE) OVER (PARTITION BY V.CLIENTE_ID,V.MES_ID) AS COSTE_SUSCRIPCION_MES,
    CASE WHEN S.COMPROMISO = 0 THEN (SUM(CANTIDAD) OVER (PARTITION BY V.CLIENTE_ID,V.MES_ID)) * 16 ELSE (SUM(CANTIDAD) OVER (PARTITION BY V.CLIENTE_ID,V.MES_ID) - S.COMPROMISO) * 12 END AS INGRESOS_EXTRA_MES,
    (SUM(CANTIDAD) OVER (PARTITION BY V.CLIENTE_ID,V.MES_ID) - S.COMPROMISO) * S.COEFICIENTE * 1.5 AS COSTE_ENTREGA_EXTRA_MES,
    (SUM(CANTIDAD) OVER (PARTITION BY V.CLIENTE_ID,V.MES_ID) - S.COMPROMISO) * AVG(V.COSTE) OVER (PARTITION BY V.CLIENTE_ID,V.MES_ID) AS COSTE_EXTRA_MES,
    CASE WHEN S.COMPROMISO = 0 THEN (SUM(CANTIDAD) OVER (PARTITION BY V.CLIENTE_ID,V.MES_ID)) * 16 ELSE (S.COMPROMISO * 9) + (SUM(CANTIDAD) OVER (PARTITION BY V.CLIENTE_ID,V.MES_ID) - S.COMPROMISO) * 12 END AS INGRESOS_TOTALES_MES,
    SUM(CANTIDAD) OVER (PARTITION BY V.CLIENTE_ID,V.MES_ID) * S.COEFICIENTE * 1.5 AS COSTE_ENTREGA_MES,
    (SUM(CANTIDAD) OVER (PARTITION BY V.CLIENTE_ID,V.MES_ID) * AVG(V.COSTE) OVER (PARTITION BY V.CLIENTE_ID,V.MES_ID)) AS COSTE_TOTAL_MES
FROM VENTA_COSTE V
LEFT JOIN SUSCRIPCION S 
    ON V.CLIENTE_ID = S.CLIENTE_ID AND S.MES_ID = V.MES_ID
),

NOMINAS AS (
    SELECT DISTINCT CAST(REPLACE(EMPLEADO_ID,'COMERCIAL ','') AS INT) AS EMPLEADO_ID, AVG(IMPORTE_NOMINA) OVER (PARTITION BY EMPLEADO_ID) + @COSTE_RENTING AS COSTE_PERSONAL 
    FROM FACT_NOMINA
    WHERE EMPLEADO_ID LIKE 'COMERCIAL%'
),

INGRESOS_CLIENTE_COMERCIAL AS (
SELECT DISTINCT F.CLIENTE_ID, SUM(INGRESOS_TOTALES_MES) OVER (PARTITION BY F.CLIENTE_ID) AS INGRESOS, C.COMERCIAL
FROM FACTURACION F
LEFT JOIN COMERCIAL_CLIENTE C 
    ON C.CLIENTE_ID = F.CLIENTE_ID
)

SELECT DISTINCT COMERCIAL,
                SUM(INGRESOS) OVER (PARTITION BY COMERCIAL) AS INGRESOS_CLIENTES,
                COUNT(*) OVER(PARTITION BY COMERCIAL) AS NUM_CLIENTES,
                N.COSTE_PERSONAL AS COSTE_COMERCIAL_MES,
                N.COSTE_PERSONAL*12 AS COSTE_COMERCIAL_ANO,
                CAST(SUM(INGRESOS) OVER (PARTITION BY COMERCIAL) / (N.COSTE_PERSONAL*12) AS DECIMAL(5,2)) AS RETORNO_EUR,
                CAST(SUM(INGRESOS) OVER (PARTITION BY COMERCIAL)*100 / @INGRESOS_TOTALES AS DECIMAL(5,2)) AS PCT_INGRESOS
FROM INGRESOS_CLIENTE_COMERCIAL C
LEFT JOIN NOMINAS N 
    ON N.EMPLEADO_ID = C.COMERCIAL
ORDER BY RETORNO_EUR DESC

(6 rows affected)

Total execution time: 00:00:10.766

COMERCIAL,INGRESOS_CLIENTES,NUM_CLIENTES,COSTE_COMERCIAL_MES,COSTE_COMERCIAL_ANO,RETORNO_EUR,PCT_INGRESOS
135,134910,167,1447,17364,"7,77","39,00"
129,85644,101,1447,17364,"4,93","25,00"
121,40353,53,"1367,2000000000003","16406,4","2,46","11,00"
116,40329,44,2112,25344,"1,59","11,00"
137,23964,27,"1553,3999999999999","18640,8","1,29","7,00"
115,12699,12,1447,17364,"0,73","3,00"


El análisis comentado nos permite extraer que entre los comerciales 135 y 129 abarcan más de la mitad de los clientes obtenidos y hasta el 65% de los ingresos totales. Por contra, entre los comerciales 137 y 115 sólo se completa el 10% de los ingresos totales y sólo 39 clientes se pueden atribuir a consecuciones de éstos. Además, el retorno del comercial 115 es inferior a 1, lo que implica que el coste de tenerlo en plantilla durante el año ha sido superior al retorno obtenido "directamente" por sus clientes a nivel de ingresos.

Por lo que conocemos sobre las zonas en las que se pretende influir y el estado de la cartera de clientes actualmente, además de por la estrategia comercial planteada, entendemos que es suficiente con tener en nómina a tres comerciales, al menos en el medio plazo, siguiendo los roles comentados en otros puntos del proyecto. Basándonos en el análisis previo, serían los comerciales 135, 129 y 121 los que permanecerían de poner en marcha el cambio en el modelo comercial que se plantea en este trabajo.

La estrategia, sin embargo, sólo podría ponerse en práctica si los datos que están en esta base de datos nos permiten obtener, mes a mes, un modelo RFM de clientes. Así, podría sustituirse la tarea presencial de "mantenimiento de clientes" por un seguimiento centralizado y automatizado que permita detectar patrones de comportamiento y, por ende, de actuación, así como señalar casos muy puntuales cada mes a los que, entonces sí, realizar un seguimiento personalizado, que podría implicar la visita comercial.

Realizaremos a continuación el modelo RFM con los datos hasta el mes actual.

## **PREPARACIÓN ESTRATEGIA (III) - Análisis RFM Clientes**

In [14]:
DECLARE @ULTIMO_MES AS NUMERIC
SET @ULTIMO_MES = (SELECT MAX(MES_ID) FROM DIM_TIEMPO);

WITH VENTA_COSTE AS (
    SELECT V.CLIENTE_ID,V.PRODUCTO_ID,V.MES_ID,V.CANTIDAD,P.COSTE FROM FACT_VENTA V
LEFT JOIN DIM_PRODUCTO P 
    ON V.PRODUCTO_ID = P.PRODUCTO_ID
),

SUSCRIPCION AS (
    SELECT *, CASE WHEN COMPROMISO <=10 THEN 1 WHEN COMPROMISO BETWEEN 10 AND 20 THEN 0.5 ELSE 0.25 END AS COEFICIENTE FROM FACT_SUBSCRIPCION
),

FACTURACION AS (
SELECT DISTINCT
    V.CLIENTE_ID,
    V.MES_ID,
    CASE WHEN S.COMPROMISO>0 THEN 1+(SUM(CANTIDAD) OVER (PARTITION BY V.CLIENTE_ID,V.MES_ID) - S.COMPROMISO) ELSE SUM(CANTIDAD) OVER (PARTITION BY V.CLIENTE_ID,V.MES_ID) - S.COMPROMISO END AS PEDIDOS,
    S.COEFICIENTE AS COEFICIENTE_COSTE_ENTREGA,
    S.COMPROMISO AS SUSCRIPCION,
    SUM(CANTIDAD) OVER (PARTITION BY V.CLIENTE_ID,V.MES_ID) - S.COMPROMISO AS EXTRA,
    SUM(CANTIDAD) OVER (PARTITION BY V.CLIENTE_ID,V.MES_ID) AS TOTAL,
    S.COMPROMISO * 9 AS INGRESOS_SUSCRIPCION_MES,
    S.COMPROMISO * S.COEFICIENTE * 1.5 AS COSTE_ENTREGA_SUB_MES,
    S.COMPROMISO * AVG(V.COSTE) OVER (PARTITION BY V.CLIENTE_ID,V.MES_ID) AS COSTE_SUSCRIPCION_MES,
    CASE WHEN S.COMPROMISO = 0 THEN (SUM(CANTIDAD) OVER (PARTITION BY V.CLIENTE_ID,V.MES_ID)) * 16 ELSE (SUM(CANTIDAD) OVER (PARTITION BY V.CLIENTE_ID,V.MES_ID) - S.COMPROMISO) * 12 END AS INGRESOS_EXTRA_MES,
    (SUM(CANTIDAD) OVER (PARTITION BY V.CLIENTE_ID,V.MES_ID) - S.COMPROMISO) * S.COEFICIENTE * 1.5 AS COSTE_ENTREGA_EXTRA_MES,
    (SUM(CANTIDAD) OVER (PARTITION BY V.CLIENTE_ID,V.MES_ID) - S.COMPROMISO) * AVG(V.COSTE) OVER (PARTITION BY V.CLIENTE_ID,V.MES_ID) AS COSTE_EXTRA_MES,
    CASE WHEN S.COMPROMISO = 0 THEN (SUM(CANTIDAD) OVER (PARTITION BY V.CLIENTE_ID,V.MES_ID)) * 16 ELSE (S.COMPROMISO * 9) + (SUM(CANTIDAD) OVER (PARTITION BY V.CLIENTE_ID,V.MES_ID) - S.COMPROMISO) * 12 END AS INGRESOS_TOTALES_MES,
    SUM(CANTIDAD) OVER (PARTITION BY V.CLIENTE_ID,V.MES_ID) * S.COEFICIENTE * 1.5 AS COSTE_ENTREGA_MES,
    (SUM(CANTIDAD) OVER (PARTITION BY V.CLIENTE_ID,V.MES_ID) * AVG(V.COSTE) OVER (PARTITION BY V.CLIENTE_ID,V.MES_ID)) AS COSTE_TOTAL_MES
FROM VENTA_COSTE V
LEFT JOIN SUSCRIPCION S 
    ON V.CLIENTE_ID = S.CLIENTE_ID AND S.MES_ID = V.MES_ID
),

DATOS_VENTA AS (
SELECT DISTINCT CLIENTE_ID,
       SUM(PEDIDOS) OVER(PARTITION BY CLIENTE_ID) AS PEDIDOS, 
       SUM(INGRESOS_TOTALES_MES) OVER (PARTITION BY CLIENTE_ID) AS TOTAL, 
       MAX(MES_ID) OVER (PARTITION BY CLIENTE_ID) AS ULTIMA_COMPRA
FROM FACTURACION
),

DATOS_RFM AS (
SELECT *, 
       CASE WHEN ULTIMA_COMPRA = @ULTIMO_MES THEN 5 ELSE NTILE(5) OVER (ORDER BY ULTIMA_COMPRA DESC) END AS R, 
       NTILE(5) OVER (ORDER BY PEDIDOS ASC) AS F, 
       NTILE(5) OVER (ORDER BY TOTAL ASC) AS M
FROM DATOS_VENTA
)

SELECT *,
       CONCAT (R,F,M) AS RFM, 
       CASE WHEN R=5 THEN 'SUSCRITO' WHEN R<>5 THEN 'NO SUSCRITO - ENVIAR FORMULARIO Y SOLICITAR FEEDBACK' END AS COMENTARIO_RECENCIA,
       CASE WHEN R=5 AND F=5 AND M=5 THEN 'CLIENTE TOP' WHEN R=5 AND (F=4 OR M=4) THEN 'CASI TOP' WHEN R=5 AND F<=3 AND M>=3 THEN 'ENVIAR PROMOCIONES/DESCUENTO POR PEDIDOS EXTRA' WHEN R=5 AND F=1 THEN 'NUEVA SUSCRIPCIÓN - ENVIAR CATÁLOGO PRODUCTO' WHEN R=5 AND M<=3 AND F>=3 THEN 'FIEL, ENVIAR CATÁLOGO PRODUCTO' WHEN R<>5 AND M<=3 THEN 'NO SUSCRITO - ENVIAR FORMULARIO Y SOLICITAR FEEDBACK' WHEN R<>5 AND M>3 THEN 'ATENCIÓN ESPECIAL - GRAN CLIENTE PERDIDO' WHEN R=5 AND F=2 THEN 'DORMIDO' ELSE 'RESTO' END AS ACTUACION
FROM DATOS_RFM 
ORDER BY RFM DESC,TOTAL DESC,PEDIDOS DESC;


WITH VENTA_COSTE AS (
    SELECT V.CLIENTE_ID,V.PRODUCTO_ID,V.MES_ID,V.CANTIDAD,P.COSTE FROM FACT_VENTA V
LEFT JOIN DIM_PRODUCTO P 
    ON V.PRODUCTO_ID = P.PRODUCTO_ID
),

SUSCRIPCION AS (
    SELECT *, CASE WHEN COMPROMISO <=10 THEN 1 WHEN COMPROMISO BETWEEN 10 AND 20 THEN 0.5 ELSE 0.25 END AS COEFICIENTE FROM FACT_SUBSCRIPCION
),

FACTURACION AS (
SELECT DISTINCT
    V.CLIENTE_ID,
    V.MES_ID,
    CASE WHEN S.COMPROMISO>0 THEN 1+(SUM(CANTIDAD) OVER (PARTITION BY V.CLIENTE_ID,V.MES_ID) - S.COMPROMISO) ELSE SUM(CANTIDAD) OVER (PARTITION BY V.CLIENTE_ID,V.MES_ID) - S.COMPROMISO END AS PEDIDOS,
    S.COEFICIENTE AS COEFICIENTE_COSTE_ENTREGA,
    S.COMPROMISO AS SUSCRIPCION,
    SUM(CANTIDAD) OVER (PARTITION BY V.CLIENTE_ID,V.MES_ID) - S.COMPROMISO AS EXTRA,
    SUM(CANTIDAD) OVER (PARTITION BY V.CLIENTE_ID,V.MES_ID) AS TOTAL,
    S.COMPROMISO * 9 AS INGRESOS_SUSCRIPCION_MES,
    S.COMPROMISO * S.COEFICIENTE * 1.5 AS COSTE_ENTREGA_SUB_MES,
    S.COMPROMISO * AVG(V.COSTE) OVER (PARTITION BY V.CLIENTE_ID,V.MES_ID) AS COSTE_SUSCRIPCION_MES,
    CASE WHEN S.COMPROMISO = 0 THEN (SUM(CANTIDAD) OVER (PARTITION BY V.CLIENTE_ID,V.MES_ID)) * 16 ELSE (SUM(CANTIDAD) OVER (PARTITION BY V.CLIENTE_ID,V.MES_ID) - S.COMPROMISO) * 12 END AS INGRESOS_EXTRA_MES,
    (SUM(CANTIDAD) OVER (PARTITION BY V.CLIENTE_ID,V.MES_ID) - S.COMPROMISO) * S.COEFICIENTE * 1.5 AS COSTE_ENTREGA_EXTRA_MES,
    (SUM(CANTIDAD) OVER (PARTITION BY V.CLIENTE_ID,V.MES_ID) - S.COMPROMISO) * AVG(V.COSTE) OVER (PARTITION BY V.CLIENTE_ID,V.MES_ID) AS COSTE_EXTRA_MES,
    CASE WHEN S.COMPROMISO = 0 THEN (SUM(CANTIDAD) OVER (PARTITION BY V.CLIENTE_ID,V.MES_ID)) * 16 ELSE (S.COMPROMISO * 9) + (SUM(CANTIDAD) OVER (PARTITION BY V.CLIENTE_ID,V.MES_ID) - S.COMPROMISO) * 12 END AS INGRESOS_TOTALES_MES,
    SUM(CANTIDAD) OVER (PARTITION BY V.CLIENTE_ID,V.MES_ID) * S.COEFICIENTE * 1.5 AS COSTE_ENTREGA_MES,
    (SUM(CANTIDAD) OVER (PARTITION BY V.CLIENTE_ID,V.MES_ID) * AVG(V.COSTE) OVER (PARTITION BY V.CLIENTE_ID,V.MES_ID)) AS COSTE_TOTAL_MES
FROM VENTA_COSTE V
LEFT JOIN SUSCRIPCION S 
    ON V.CLIENTE_ID = S.CLIENTE_ID AND S.MES_ID = V.MES_ID
),

DATOS_VENTA AS (
SELECT DISTINCT CLIENTE_ID,
       SUM(PEDIDOS) OVER(PARTITION BY CLIENTE_ID) AS PEDIDOS, 
       SUM(INGRESOS_TOTALES_MES) OVER (PARTITION BY CLIENTE_ID) AS TOTAL, 
       MAX(MES_ID) OVER (PARTITION BY CLIENTE_ID) AS ULTIMA_COMPRA
FROM FACTURACION
),

DATOS_RFM AS (
SELECT *, 
       CASE WHEN ULTIMA_COMPRA = @ULTIMO_MES THEN 5 ELSE NTILE(5) OVER (ORDER BY ULTIMA_COMPRA DESC) END AS R, 
       NTILE(5) OVER (ORDER BY PEDIDOS ASC) AS F, 
       NTILE(5) OVER (ORDER BY TOTAL ASC) AS M
FROM DATOS_VENTA
),

RFM_FINAL AS (
    SELECT *,
       CONCAT (R,F,M) AS RFM, 
       CASE WHEN R=5 THEN 'SUSCRITO' WHEN R<>5 THEN 'NO SUSCRITO - ENVIAR FORMULARIO Y SOLICITAR FEEDBACK' END AS COMENTARIO_RECENCIA,
       CASE WHEN R=5 AND F=5 AND M=5 THEN 'CLIENTE TOP' WHEN R=5 AND (F=4 OR M=4) THEN 'CASI TOP' WHEN R=5 AND F<=3 AND M>=3 THEN 'ENVIAR PROMOCIONES/DESCUENTO POR PEDIDOS EXTRA' WHEN R=5 AND F=1 THEN 'NUEVA SUSCRIPCIÓN - ENVIAR CATÁLOGO PRODUCTO' WHEN R=5 AND M<=3 AND F>=3 THEN 'FIEL, ENVIAR CATÁLOGO PRODUCTO' WHEN R<>5 AND M<=3 THEN 'NO SUSCRITO - ENVIAR FORMULARIO Y SOLICITAR FEEDBACK' WHEN R<>5 AND M>3 THEN 'ATENCIÓN ESPECIAL - GRAN CLIENTE PERDIDO' WHEN R=5 AND F=2 THEN 'DORMIDO' ELSE 'RESTO' END AS ACTUACION
    FROM DATOS_RFM 
)

SELECT DISTINCT RFM, COUNT(*) OVER(PARTITION BY RFM) AS NUM_CLIENTES, COMENTARIO_RECENCIA, ACTUACION 
FROM RFM_FINAL
ORDER BY NUM_CLIENTES DESC

(404 rows affected)

(15 rows affected)

Total execution time: 00:00:00.563

CLIENTE_ID,PEDIDOS,TOTAL,ULTIMA_COMPRA,R,F,M,RFM,COMENTARIO_RECENCIA,ACTUACION
Cliente - 275,57,2034,202004,5,5,5,555,SUSCRITO,CLIENTE TOP
Cliente - 066,45,1944,202004,5,5,5,555,SUSCRITO,CLIENTE TOP
Cliente - 240,42,1854,202004,5,5,5,555,SUSCRITO,CLIENTE TOP
Cliente - 201,50,1815,202004,5,5,5,555,SUSCRITO,CLIENTE TOP
Cliente - 400,39,1809,202004,5,5,5,555,SUSCRITO,CLIENTE TOP
Cliente - 116,44,1797,202004,5,5,5,555,SUSCRITO,CLIENTE TOP
Cliente - 337,46,1785,202004,5,5,5,555,SUSCRITO,CLIENTE TOP
Cliente - 084,48,1782,202004,5,5,5,555,SUSCRITO,CLIENTE TOP
Cliente - 290,44,1770,202004,5,5,5,555,SUSCRITO,CLIENTE TOP
Cliente - 190,43,1740,202004,5,5,5,555,SUSCRITO,CLIENTE TOP


RFM,NUM_CLIENTES,COMENTARIO_RECENCIA,ACTUACION
555,71,SUSCRITO,CLIENTE TOP
511,61,SUSCRITO,NUEVA SUSCRIPCIÓN - ENVIAR CATÁLOGO PRODUCTO
544,60,SUSCRITO,CASI TOP
533,47,SUSCRITO,ENVIAR PROMOCIONES/DESCUENTO POR PEDIDOS EXTRA
522,45,SUSCRITO,DORMIDO
532,21,SUSCRITO,"FIEL, ENVIAR CATÁLOGO PRODUCTO"
521,19,SUSCRITO,DORMIDO
523,17,SUSCRITO,ENVIAR PROMOCIONES/DESCUENTO POR PEDIDOS EXTRA
512,15,SUSCRITO,NUEVA SUSCRIPCIÓN - ENVIAR CATÁLOGO PRODUCTO
534,12,SUSCRITO,CASI TOP


El modelo RFM construido en la consulta previa muestra, en primer lugar, lo que ya habíamos intuido previamente en el trabajo en lo referente a las suscripciones: todos los clientes suscritos en algún momento del año estaban presentes en abril de 2020 -por ello, todos tienen recencia=5-. Además, vemos que el caso más común es el 555 -CLIENTE TOP- (71). Más de la mitad de los clientes existentes son TOP (71), CASI TOP (102) o NUEVOS (76), lo que nos deja una cartera de clientes que, si bien advertíamos que está geográficamente distribuida de forma demasiado uniforme, goza de una muy buena salud.

En el modelo se crean parámetros de actuación en casos de pérdida de suscripción o de caída en valor monetario de clientes con alta frecuencia. Estas dos tipologías citadas se encontrarían entre aquellas que necesitarían de una mayor atención (o más personalizada). Para la mayoría del resto de casos se fijan parámetros de actuación genéricos que, de ser bien aplicados, asegurarían el mantenimiento o ascenso del cliente con un coste comercial y operativo muy inferior al existente hasta hoy.

## **CONCLUSIONES**

Citando de nuevo lo planteado durante este proyecto, se concluye que deben limitarse las funciones de los comerciales a la captación de nuevos clientes y, de modo muy puntual, a la atención al cliente existente si nuestro modelo RFM establece que éste está en peligro de perderse (por una reducción mes a mes en su compromiso de ensaladas). <u>Para un área total inferior a la extensión de Galicia es excesivo que existan seis comerciales, especialmente si tenemos en cuenta que la mitad del área total tiene un grado de de fidelización alto.</u> Por ello, consideramos que, desde dirección, deben impulsarse los siguientes cambios:

> · <u>Sustitución del seguimiento presencial de clientes que hasta ahora realizaban los comerciales por una gestión basada en el análisis y tratamiento de los datos y una estrategia de posicionamiento en redes y canales de distribución (apps como JustEat o UberEats). La operativa comercial se centrará casi exclusivamente en la captación de clientes nuevos.</u>
> 
> · <u>La operativa comercial debe centrarse en la búsqueda de nuevos clientes en la zona con mayor margen de crecimiento (ZONA\_ID= 4 5 y 6). Se destinarán dos comerciales a trabajar única y exclusivamente en aportar nuevas altas de estas tres zonas. Habrá un tercer comercial adicional que realizará parcialmente esta labor principal y, además, trabajará el mantenimiento de clientes acorde al segundo punto.</u>
> 
> · <u>En las zonas 1, 2 y 3 la operativa estará orientada en una atención al cliente "en peligro de perderse" -cuya cantidad se entiende que siempre será muy limitada-, que serán identificados por el modelo RFM en los informes mensuales que procedan de la base de datos. Un único comercial podrá dedicar un % de su tiempo a esta labor, y el resto a lo tocante en el punto anterior.</u>

Así, se prescindiría de tres de los comerciales (basándonos en el análisis de la red comercial, serían los comerciales 116, 137 y 115), mientras que cobraría importancia a nivel estructural la parte de gestión y análisis de datos, cuya labor corresponde a Christian, según el enunciado. Tanto Brais como Christian, además de los tres comerciales y el cocinero, percibirían nómina a final de mes. Además, se sumarían costes, al menos a corto/medio plazo en aspectos ya mencionados como la inversión en posicionamiento en redes y el establecimiento del negocio en algún distribuidor. A medio plazo el crecimiento del negocio puede llevarnos también a necesitar un segundo cocinero.&nbsp;  En ningún caso estas inversiones superarían lo ahorrado en costes con los ajustes comentados, mientras que, bajo los supuestos señalados, es previsible un aumento en el número de clientes, ingresos y margen neto.